In [5]:
import pandas as pd
import numpy as np
import random

#get the 'df' row where the index contains 'string'
#PARAMS
#df: dataframe
#string: string
def get_row(df, string):
    return pd.Series(df.index).str.contains(string.strip(), case=False)

def random_schedule_day(dframe, length):
    sample = random.sample(range(0,len(dframe.columns)), length)
    return [dframe.columns[idx] for idx in sample]

#get the 'df' column where the index contains 'string'
#PARAMS
#df: dataframe
#string: string
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string.strip(), case=False)

#calculate the difference in minutes between two given times expressed as HH:MM
#PARAMS
#time_table: list
#day: string
#index, index2: integer
def difference(time_table, day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

#get the name expressed in the index of dataframe 'dft' that contains 'nome'
#PARAMS
#dft: dataframe
#nome: string
def getName(df, nome):
    if nome.lower() == "crema":
        nome ="crema,"
    city = df[get_row(df, nome).values].index[0]
    return city

#returns a pd.Series where keys are pair of destinations' names and the value is the time to go
#from the first city to the second
#PARAMS
#schedule: lists of cities
#dframe: dataframe
#RETURNS: city1 $ city2 -> time
def new_route(schedule, dframe):
    times = []
    keys = []
    
    row = dframe[get_row(dframe, schedule[0]).values]
    key1 = ''
    key2 = ''
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        paese=getName(dframe, schedule[i])
        paese=paese.strip()
        if 'Martignana' in paese:
            paese = 'Martignana di Po'
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        try:
            value=row.loc[:,get_column(dframe, paese).values].values[0][0]
        except:
            print(row.loc[:,get_column(dframe, paese).values])
            print(paese)
            return
        times.append(round(value))
        row = dframe[get_row(dframe, paese).values]
        try:
            key2 = row.index[0]
        except:
            raise Exception("ciao")
        keys.append(key1 + ' $ ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

#returns a list of cities from a pd.Series in the format obtained by 'new_route'
#PARAMS
#serie: pd.Series
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        splitted = keys[i].split("$")
        cities = cities + [splitted[0].strip()]
    cities = cities + [splitted[1].strip()]
    return cities 

#returns the distances expressed in dft from 'start' to 'end'
#PARAMS
#start, end: string
#dft: dataframe
def timeDistance(start, end, dft):
    start_city_row = dft[get_row(dft, start.strip()).values]
    return start_city_row.loc[:,get_column(dft, end).values].values[0][0]

#given a city 'to_insert' returns the pd.Serie where the keys are pair of cities and values are the time to go
#from the first city to 'to_insert' plus 'to_insert' to second city
#PARAMS
#steps_times: pd.Series (new_route output)
#to_insert: string
#dframe: dataframe
def find_best_position(steps_times, to_insert, dframe):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('$')
        if(cities[0].strip()=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            row1 = dframe[get_row(dframe, 'San Bassano, Mons. Frosi').values]
        else:
            row1 = dframe[get_row(dframe, cities[0].strip()).values]
        f_c1_to_curr = row1.loc[:,get_column(dframe, to_insert).values].values[0][0]
        row2 = dframe[get_row(dframe, to_insert).values]
        f_curr_to_c2 = row2.loc[:,get_column(dframe, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

#returns the index in lista where to put 'to_insert' using find_best_position
#PARAMS
#lista: list
#schedule: pd.Series
#to_insert: string
#dframe: dataframe
def best_index(lista, schedule, to_insert, dframe):
    idx = find_best_position(schedule, to_insert, dframe).idxmin()
    cities = idx.split("$")
    a=cities[0].strip()
    new_lista = [obj.lower() for obj in lista]
    try:
        #print("classic return")
        return new_lista.index(a.lower())+1
    except:
        #problema nome diverso
        print("Nome strano", a, sep="  ")
        if(a.lower() == 'casale cremasco'):
            a = 'Casale Cremasco-Vidolasco, Roma, 56'
            return new_lista.index(a)+1
        elif(a.lower() == 'gadesco'):
            a = 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
            return new_lista.index(a)+1
        elif(a=="San Bassano, Mons. Frosi (ex piazza Scuole)"):
            a = "San Bassano, Mons. Frosi"
            return new_lista.index(a)+1
        
def check_cities(df1_fur1, df1_fur2, df2_fur1, df2_fur2):
    l1 = []
    for day in df1_fur1.columns:
        l1 = l1 + list(df1_fur1[day].dropna())
    for day in df1_fur2.columns:
        l1 = l1 + list(df1_fur2[day].dropna())
    l2 = []
    for day in df2_fur1.columns:
        l2 = l2 + list(df2_fur1[day].dropna())
    for day in df2_fur2.columns:
        l2 = l2 + list(df2_fur2[day].dropna())
        
    l1.sort()
    l2.sort()
    
    li_dif = [i for i in l1 if i not in l2]
    li_dif += [i for i in l2 if i not in l1]
    l1_freq = [(x, l1.count(x)) for x in l1]
    l2_freq = [(x, l2.count(x)) for x in l2]
    
    return li_dif, l1_freq, l2_freq

def translate_name(dframe, fur):
    df = pd.DataFrame({})
    for d in fur.columns:
        df1 = pd.DataFrame([getName(dframe, x) for x in list(fur[d].dropna())], columns = [d])
        df = pd.concat([df,df1], ignore_index=False, axis=1)
    return df

In [122]:
import pandas as pd
dft = pd.read_csv("../Dati/distanze/corrected_times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
greedy1 = pd.read_csv("../Dati/turni_f/original_start/f1_original.csv", delimiter=";", encoding = "ISO-8859-1")
greedy2 = pd.read_csv("../Dati/turni_f/original_start/f2_original.csv", delimiter=";", encoding = "ISO-8859-1")
ls1 = pd.read_csv("../Dati/turni_f/original_start/original_ls1.csv", delimiter=";", encoding = "ISO-8859-1")
ls2 = pd.read_csv("../Dati/turni_f/original_start/original_ls2.csv", delimiter=";", encoding = "ISO-8859-1")
opt1 = pd.read_csv("../Dati/turni_f/original_start/original_2opt1.csv", delimiter=";", encoding = "ISO-8859-1")
opt2 = pd.read_csv("../Dati/turni_f/original_start/original_2opt2.csv", delimiter=";", encoding = "ISO-8859-1")

week_greedy = 0
for d in greedy1.columns:
    week_greedy += sum(new_route(greedy1[d].dropna(), dft).values)
for d in greedy2.columns:
    week_greedy += sum(new_route(greedy2[d].dropna(), dft).values)

week_ls = 0
for d in ls1.columns:
    week_ls += sum(new_route(ls1[d].dropna(), dft).values)
for d in ls2.columns:
    week_ls += sum(new_route(ls2[d].dropna(), dft).values)

impr_greedy_ls = round((week_greedy-week_ls)/week_ls, 3)


week_opt = 0
for d in opt1.columns:
    week_opt += sum(new_route(opt1[d].dropna(), dft).values)
for d in opt2.columns:
    week_opt += sum(new_route(opt2[d].dropna(), dft).values)
    
impr_ls_opt = round((week_ls-week_opt)/week_opt,3)

print(impr_greedy_ls,impr_ls_opt)

week = list(ls1.columns)
df1cols = week[:]
df1cols.append("tot")
sweek = list(ls2.columns)
df2cols = sweek[:]
df2cols.append("tot")
df1 = pd.DataFrame({}, columns = df1cols,  index = ["original", "ls", "2opt", "original-ls", "ls-2opt"])
df2 = pd.DataFrame({}, columns = df2cols, index = ["original", "ls", "2opt", "original-ls", "ls-2opt"])


for d in week:
    df1.loc["original", d] = sum(new_route(greedy1[d].dropna(), dft).values)
    df1.loc["ls", d] = sum(new_route(ls1[d].dropna(), dft).values)
    df1.loc["2opt", d] = sum(new_route(opt1[d].dropna(), dft).values)
for d in sweek:
    df2.loc["original", d] = sum(new_route(greedy2[d].dropna(), dft).values)
    df2.loc["ls", d] = sum(new_route(ls2[d].dropna(), dft).values)
    df2.loc["2opt", d] = sum(new_route(opt2[d].dropna(), dft).values)
    
for d in df1.index:
    df1.loc[d,"tot"] = sum(df1.loc[d][week])
for d in df2.index:
    df2.loc[d,"tot"] = sum(df2.loc[d][sweek])
    
for d in df1.columns:
    df1.loc["original-ls", d] = round((df1.loc["original", d] - df1.loc["ls", d])/df1.loc["ls", d], 3)
    df1.loc["ls-2opt", d] = round((df1.loc["ls", d] - df1.loc["2opt", d])/df1.loc["2opt", d], 3)
for d in df2.columns:
    df2.loc["original-ls", d] = round((df2.loc["original", d] - df2.loc["ls", d])/df2.loc["ls", d], 3)
    df2.loc["ls-2opt", d] = round((df2.loc["ls", d] - df2.loc["2opt", d])/df2.loc["2opt", d], 3)

tot = pd.DataFrame({}, columns = ["cost"], index = ["original", "ls", "2opt", "original-ls", "ls-2opt"])
for i in ["original", "ls", "2opt"]:
    tot.loc[i,"cost"] = df1.loc[i, "tot"] + df2.loc[i, "tot"]
for i in ["original-ls", "ls-2opt"]:
    tot.loc["original-ls", "cost"] = round(float((tot.loc["original"] - tot.loc["ls", "cost"])/tot.loc["ls", "cost"]),3)
    tot.loc["ls-2opt", "cost"] = round(float((tot.loc["ls"] - tot.loc["2opt", "cost"])/tot.loc["2opt", "cost"]), 3)

0.133 0.093


In [128]:
df1

,lunedi,martedi,mercoledi,giovedi,venerdi,tot
original,235,246,283,261,239,1264
ls,197,224,219,227,216,1083
2opt,187,210,199,174,191,961
original-ls,0.193,0.098,0.292,0.15,0.106,0.167
ls-2opt,0.053,0.067,0.101,0.305,0.131,0.127


In [124]:
df1.to_csv("../Dati/altro/original_improvement_table1", sep=";", encoding = "ISO-8859-1")
df2.to_csv("../Dati/altro/original_improvement_table2", sep=";", encoding = "ISO-8859-1")
tot.to_csv("../Dati/altro/original_improvement_tot", sep=";", encoding = "ISO-8859-1")